## Regression in panel.Tabulator

Minimal, complete, and reproducible example to demonstrate a regression in the `Tabulator` widget in `panel>0.12.1`.

In [1]:
import pandas as pd
import panel as pn

pn.extension('tabulator')

In [2]:
{m.__name__: m.__version__ for m in (pd, pn)}

{'pandas': '1.3.5', 'panel': '0.13.0a31'}

## Create a Tabulator widget from a DataFrame

Code and description columns are both string (object) dtypes. Descriptions are random phrases from an [online generator][rwg].

[rwg]: https://randomwordgenerator.com/phrase.php

In [3]:
descr = [
    'Under the Weather',
    'Top Drawer',
    'Happy as a Clam',
    'Cut To The Chase',
    'Knock Your Socks Off',
    'A Cold Day in Hell',
    'All Greek To Me',
    'A Cut Above',
    'Cut The Mustard',
    'Up In Arms',
    'Playing For Keeps',
    'Fit as a Fiddle',
]

In [4]:
code = [f'{i:02d}' for i in range(len(descr))]

In [5]:
df = pd.DataFrame(
    dict(
        code=code,
        descr=descr
    )
)

In [6]:
tbl = pn.widgets.Tabulator(
    df,
    pagination='local',
    page_size=6,
    selectable='checkbox',
    show_index=False,
    height=300,
    width=400
)

## Filter the Tabulator

Code adapted from the [Function based filtering][fbf] section of the [Tabulator][tbl] page of the [Reference Gallery][rg].

[fbf]: https://panel.holoviz.org/reference/widgets/Tabulator.html#function-based-filtering
[tbl]: https://panel.holoviz.org/reference/widgets/Tabulator.html#widgets-gallery-tabulator
[rg]: https://panel.holoviz.org/reference/index.html

In [7]:
filter_wgts = dict(
    code=pn.widgets.TextInput(name='code', width=100),
    descr=pn.widgets.TextInput(name='descr', width=180)
)

In [8]:
def contains_filter(df, pattern, column, case=False, regex=False):
    if not pattern:
        return df
    idx = df[column].str.contains(
        pattern,
        case=case,
        regex=regex
    )
    return df[idx]

In [9]:
for col in df.columns:
    tbl.add_filter(
        pn.bind(
            contains_filter,
            pattern=filter_wgts[col],
            column=col
        )
    )

## Add selected rows to a 2nd Tabulator

In [10]:
# Set dtypes by inferring from a single row:
df2_template = pd.DataFrame(
    data=[['c', 'd', 1.0]],
    columns=['code', 'descr', 'temp'],
)

In [11]:
# Discard the dummy row to create the empty DataFrame:
df2 = df2_template.head(0)

In [12]:
tbl2 = pn.widgets.Tabulator(
    df2,
    pagination='local',
    page_size=6,
    # selectable='checkbox',
    show_index=False,
    height=300,
    width=400
)

In [13]:
def fill_tbl2(event):
    if tbl.selection == []:
        return
    df_new = (
        df2.append(
            other=tbl.selected_dataframe,
            ignore_index=True
        )
        .fillna(
            df2_template.to_dict('records')[0]
        )
    )
    tbl2.value = tbl2.value.append(df_new).reset_index(drop=True)

In [14]:
btn_fill_tbl2 = pn.widgets.Button(
    name='Add selected rows',
    button_type='primary',
    width=100,
)

btn_fill_tbl2.on_click(fill_tbl2)

## Create and display the UI

In [15]:
ui = pn.Column(
    pn.Row(*filter_wgts.values()),
    tbl,
    # btn_fill_tbl2,
    # tbl2,
)

In [16]:
ui.servable()

Column
    [0] Row
        [0] TextInput(name='code', width=100)
        [1] TextInput(name='descr', width=180)
    [1] Tabulator(height=300, page_size=6, pagination='local', selectable='checkbox', show_index=False, sizing_mode='fixed', value=   code       ..., width=400)

In [17]:
tbl.selection

[0, 1, 2]

In [18]:
tbl.current_view

,code,descr
3,03,Cut To The Chase
7,07,A Cut Above
8,08,Cut The Mustard


In [19]:
tbl._filter_dataframe(tbl.value).index.tolist()

[3, 7, 8]

In [20]:
tbl.selected_dataframe

,code,descr
3,03,Cut To The Chase
7,07,A Cut Above
8,08,Cut The Mustard


## Demonstrate the bug for panel>0.12.1

Show that adding rows to `tbl2` from rows selected in `tbl` works when `tbl` has no active filtering, but fails (for panel>0.12.1) when `tbl` is filtered according to the `TextInput` widgets.

In [ ]:
demo = True

In [ ]:
filter_wgts['descr'].value = 'cut'

In [ ]:
tbl.selection

In [ ]:
tbl.current_view

In [ ]:
tbl.selection = tbl.current_view.index.to_list()

In [ ]:
tbl.selection

In [ ]:
tbl.selected_dataframe

In [ ]:
# This works for all panel versions:
if demo:
    tbl.selection = [0, 1]
    fill_tbl2(True)
    tbl.selection = []
    filter_wgts['descr'].value = 'cut'
    try:
        tbl.selection = tbl.current_view.index.to_list()
    except AttributeError: # panel=0.12.1
        tbl.selection = tbl._filter_dataframe(tbl.value).index.to_list()

In [ ]:
# This fails for panel>0.12.1:
if demo:
    fill_tbl2(True)

In [ ]:
tbl2

In [ ]:
tbl.selection

In [ ]:
tbl

In [ ]:
tbl.current_view.index.to_list()

In [ ]:
tbl._filter_dataframe(tbl.value).index.to_list()

In [ ]:
tbl.selection = tbl.current_view.index.to_list()

In [ ]:
tbl.selection